In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pycountry



## Get Top Places to Visit

In [9]:
def getSoup(link):
    r = requests.get(link)
    r.encoding = 'UTF-8'
    
    return BeautifulSoup(r.text,'lxml')


### Links


GETTING LIST OF COUNTRIES NAMES

In [3]:
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2.lower()


In [6]:
countries_names = [countries.keys()]


TRIPADVISOR LINKS FOR EVERY COUNTRY

In [7]:
links_google= ["https://www.google.com/search?q=tripadvisor+15+best+things+to+do+in+" + x for x in countries.keys()]


#### Country Things to do top 15

just fucntion

In [10]:
soup = getSoup('https://www.google.com/search?q=tripadvisor+best+things+to+do+in+slovakia')
h3=soup.find('h3', {'class':"r"})
print(h3)

<h3 class="r"><a href="/url?q=https://www.tripadvisor.cz/Attractions-g274922-Activities-Slovakia.html&amp;sa=U&amp;ved=0ahUKEwjk-Ifgl9fhAhUhCTQIHRutCXYQFggUMAA&amp;usg=AOvVaw3uZpKEvOPqNKkGFSz1cEZZ">10 nejlepš�ch aktivit a pam�tek - <b>Slovensko</b> - <b>TripAdvisor</b></a></h3>


In [11]:
soup = getSoup('https://www.google.com/search?q=tripadvisor+best+things+to+do+in+denmark')
h3=soup.findAll('h3', {'class':"r"} )[2]
a=h3.contents[0]
url=a.get('href')
url=url.partition("Attractions")[2].partition(".html")[0]
ta = 'http://tripadvisor.com/Attractions' 
adress= ta+url
print(adress)



http://tripadvisor.com/Attractions-g189512-Activities-Denmark


In [12]:
a=h3.contents[0]
url=a.get('href')
url=url.partition("Attractions")[2].partition(".html")[0] #tripadvisor.cz
ta = 'http://tripadvisor.com/Attractions' 
adress= ta+url
print(adress)

http://tripadvisor.com/Attractions-g189512-Activities-Denmark


for loop

In [32]:
links_tripadvisor=[]
for i in links_google:
    soup = getSoup(i)
    h3=soup.find('h3', {'class':"r"})
    a=h3.contents[0]
    url=a.get('href')
    url=url.partition("Attractions")[2].partition(".html")[0]
    ta = 'http://tripadvisor.com/Attractions' 
    adress= ta+url
    links_tripadvisor.append(adress)
print(links_tripadvisor)

['http://tripadvisor.com/Attractions', 'http://tripadvisor.com/Attractions-g293828-Activities-Rwanda', 'http://tripadvisor.com/Attractions-g294481-Activities-Tuvalu', 'http://tripadvisor.com/Attractions', 'http://tripadvisor.com/Attractions-g294307-Activities-Ecuador', 'http://tripadvisor.com/Attractions-g188553-Activities-The_Netherlands', 'http://tripadvisor.com/Attractions-g1767153-Activities-St_Helena_Island', 'http://tripadvisor.com/Attractions-g293843-Activities-Lusaka_Lusaka_Province', 'http://tripadvisor.com/Attractions', 'http://tripadvisor.com/Attractions', 'http://tripadvisor.com/Attractions-g293753-Activities-Tunisia', 'http://tripadvisor.com/Attractions-g147400-Activities-U_S_Virgin_Islands', 'http://tripadvisor.com/Attractions', 'http://tripadvisor.com/Attractions-g293961-Activities-Sri_Lanka', 'http://tripadvisor.com/Attractions-g293810-Activities-Malawi', 'http://tripadvisor.com/Attractions-g293747-Activities-Tanzania', 'http://tripadvisor.com/Attractions-g294064-Activi

### Attraction Titles 


In [35]:
attraction_titles_all=[]
for i in links_tripadvisor:    
    soup = getSoup(i)
    divs = soup.findAll('div', {'class':'listing_title'}) 
    text_chunks = {div.contents[1] for div in divs}
    attraction_titles_all.append({title.contents[0] for title in text_chunks})
    

IBA KÓD

In [16]:

soup = getSoup("http://tripadvisor.com/Attractions-g293961-Activities-Sri_Lanka")
divs = soup.findAll('div', {'class':'listing_title'}) 
text_chunks = {div.contents[1] for div in divs}
attraction_titles = {title.contents[0] for title in text_chunks}
print(attraction_titles)

{'Euphoria Spice and Herbal', 'Gangaramaya (Vihara) Buddhist Temple', 'Bentota Beach', 'Jami Ul-Alfar Mosque', 'Hikkaduwa Beach', 'Mulgirigala Raja Maha Vihara', 'Main Fish Market', 'Temple of the Sacred Tooth Relic', 'Udawalawe National Park', 'Muthurajawela Marsh', 'Yala National Park', 'Sinharaja Forest Reserve', 'Old Town of Galle and its Fortifications', 'Dutch Canal', 'Horton Plains National Park', 'Pidurangala Rock', 'Negombo Lagoon', 'Ancient City of Sigiriya', 'Negombo Fish Market', 'Mount Lavinia Beach', 'Kelaniya Raja Maha Vihara', "Little Adam's Peak", 'Mirissa Beach', 'Colombo National Museum', 'Japanese Peace Pagoda', 'Angurukaramulla Temple', 'Galle Fort', 'Koneswaram Temple', 'Sea Turtle Farm', "St. Mary's Church"}


## CLASS

In [ ]:
class Country:
    '''
    '''
    
    def __init__(self,link):
        self.link = link
        r = requests.get(link)
        r.encoding='UTF-8'
        self.soup = BeautifulSoup(r.text,'lxml')
    
    
    